<a href="https://colab.research.google.com/github/mafe230604/ejercicio_basicos_colab/blob/main/condicionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTRUCCIONES CONDICIONALES 
  

*   if
*   if else
*   if elif else   







In [ ]:
"""Ejercicio1:
Programa para verificar si una persona 
es mayor de edad"""

print("--------------------------------")
print("---------mayor de edad ---------")
print("--------------------------------")

#input
edad=int(input("Digite la edad:"))

#processing
if edad >= 18:

    print("Lapersona es MAYOR DE EDAD ")
else:
    print("La persona es MENOR DE EDAD")

In [ ]:
import pygame
import sys    # para usar exit()
import time   # para usar sleep()

ANCHO = 640 # Ancho de la pantalla.
ALTO = 480  # Alto de la pantalla.
color_rosado= (255, 255, 255)  # Color azul para el fondo.
color_blanco= (0, 0, 0) # Color blanco, para textos.


class Escena:
    "Esqueleto para cada una de las escenas del videojuego."
    def __init__(self):
        self.proximaEscena = False
        self.jugando = True

    def leer_eventos(self, eventos):
        "Lee los eventos para interactuar con los objetos."
        pass

    def actualizar(self):
        "Actualiza los objetos en la pantalla."
        pass

    def dibujar(self, pantalla):
        "Dibuja los objetos en la pantalla."
        pass

    def cambiar_escena(self, escena):
        "Cambia la escena del juego."
        self.proximaEscena = escena

class Director:
    def __init__(self, titulo = "", res = (ANCHO, ALTO)):
        pygame.init()
        # Inicializando pantalla.
        self.pantalla = pygame.display.set_mode(res)
        # Configurar título de pantalla.
        pygame.display.set_caption(titulo)
        # Crear el reloj.
        self.reloj = pygame.time.Clock()
        self.escena = None
        self.escenas = {}

    def ejecutar(self, escena_inicial, fps = 60):
        self.escena = self.escenas[escena_inicial]
        jugando = True
        while jugando:
            self.reloj.tick(fps)
            eventos = pygame.event.get()
            # Revisar todos los eventos.
            for evento in eventos:
                # Si se presiona la tachita de la barra de título,
                if evento.type == pygame.QUIT:
                    # cerrar el videojuego.
                    jugando = False

            self.escena.leer_eventos(eventos)
            self.escena.actualizar()
            self.escena.dibujar(self.pantalla)

            self.elegirEscena(self.escena.proximaEscena)

            if jugando:
                jugando = self.escena.jugando

            pygame.display.flip()

        time.sleep(3)

    def elegirEscena(self, proximaEscena):
        if proximaEscena:
            if proximaEscena not in self.escenas:
                self.agregarEscena(proximaEscena)
            self.escena = self.escenas[proximaEscena]

    def agregarEscena(self, escena):
        escenaClase = 'Escena'+escena
        escenaObj = globals()[escenaClase]
        self.escenas[escena] = escenaObj();


class EscenaNivel1(Escena):
    def __init__(self):
        Escena.__init__(self)

        self.bolita = Bolita()
        self.jugador = Paleta()
        self.muro = Muro(50)

        self.puntuacion = 0
        self.vidas = 3
        self.esperando_saque = True

        # Ajustar repetición de evento de tecla presionada.
        pygame.key.set_repeat(30)

    def leer_eventos(self, eventos):
        for evento in eventos:
            if evento.type == pygame.KEYDOWN:
                self.jugador.update(evento)
                if self.esperando_saque == True and evento.key == pygame.K_SPACE:
                    self.esperando_saque = False
                    if self.bolita.rect.centerx < ANCHO / 2:
                        self.bolita.speed = [3, -3]
                    else:
                        self.bolita.speed = [-3, -3]

    def actualizar(self):
        # Actualizar posición de la bolita.
        if self.esperando_saque == False:
            self.bolita.update()
        else:
            self.bolita.rect.midbottom = self.jugador.rect.midtop

        # Colisión entre bolita y jugador.
        if pygame.sprite.collide_rect(self.bolita, self.jugador):
            self.bolita.speed[1] = -self.bolita.speed[1]

        # Colisión de la bolita con el muro.
        lista = pygame.sprite.spritecollide(self.bolita, self.muro, False)
        if lista:
            ladrillo = lista[0]
            cx = self.bolita.rect.centerx
            if cx < ladrillo.rect.left or cx > ladrillo.rect.right:
                self.bolita.speed[0] = -self.bolita.speed[0]
            else:
                self.bolita.speed[1] = -self.bolita.speed[1]
            self.muro.remove(ladrillo)
            self.puntuacion += 10

        # Revisar si bolita sale de la pantalla.
        if self.bolita.rect.top > ALTO:
            self.vidas -= 1
            self.esperando_saque = True

        if self.vidas <= 0:
            self.cambiar_escena('JuegoTerminado')

    def dibujar(self, pantalla):
        # Rellenar la pantalla.
        pantalla.fill(color_rosado)
        # Mostrar puntuación
        self.mostrar_puntuacion(pantalla)
        # Mostrar vidas.
        self.mostrar_vidas(pantalla)
        # Dibujar bolita en pantalla.
        pantalla.blit(self.bolita.image, self.bolita.rect)
        # Dibujar jugador en pantalla.
        pantalla.blit(self.jugador.image, self.jugador.rect)
        # Dibujar los ladrillos.
        self.muro.draw(pantalla)

    def mostrar_puntuacion(self, pantalla):
        fuente = pygame.font.SysFont('Consolas', 20)
        texto = fuente.render(str(self.puntuacion).zfill(5), True, color_blanco)
        texto_rect = texto.get_rect()
        texto_rect.topleft = [0, 0]
        pantalla.blit(texto, texto_rect)

    def mostrar_vidas(self, pantalla):
        fuente = pygame.font.SysFont('Consolas', 20)
        cadena = "Vidas: " + str(self.vidas).zfill(2)
        texto = fuente.render(cadena, True, color_blanco)
        texto_rect = texto.get_rect()
        texto_rect.topright = [ANCHO, 0]
        pantalla.blit(texto, texto_rect)

class EscenaJuegoTerminado(Escena):
    def actualizar(self):
        self.jugando = False

    def dibujar(self, pantalla):
        fuente = pygame.font.SysFont('Arial', 72)
        texto = fuente.render('Juego terminado :(', True, color_blanco)
        texto_rect = texto.get_rect()
        texto_rect.center = [ANCHO / 2, ALTO / 2]
        pantalla.blit(texto, texto_rect)


class Bolita(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        # Cargar imagen
        self.image = pygame.image.load('bolita.png')
        # Obtener rectángulo de la imagen
        self.rect = self.image.get_rect()
        # Posición inicial centrada en pantalla.
        self.rect.centerx = ANCHO / 2
        self.rect.centery = ALTO / 2
        # Establecer velocidad inicial.
        self.speed = [3, 3]

    def update(self):
        # Evitar que salga por debajo.
        if self.rect.top <= 0:
            self.speed[1] = -self.speed[1]
        # Evitar que salga por la derecha.
        elif self.rect.right >= ANCHO or self.rect.left <= 0:
            self.speed[0] = -self.speed[0]
        # Mover en base a posición actual y velocidad.
        self.rect.move_ip(self.speed)

class Paleta(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        # Cargar imagen
        self.image = pygame.image.load('paleta.png')
        # Obtener rectángulo de la imagen
        self.rect = self.image.get_rect()
        # Posición inicial centrada en pantalla en X.
        self.rect.midbottom = (ANCHO / 2, ALTO - 20)
        # Establecer velocidad inicial.
        self.speed = [0, 0]

    def update(self, evento):
        # Buscar si se presionó flecha izquierda.
        if evento.key == pygame.K_LEFT and self.rect.left > 0:
            self.speed = [-5, 0]
        # Si se presionó flecha derecha.
        elif evento.key == pygame.K_RIGHT and self.rect.right < ANCHO:
            self.speed = [5, 0]
        else:
            self.speed = [0, 0]
        # Mover en base a posición actual y velocidad.
        self.rect.move_ip(self.speed)

class Ladrillo(pygame.sprite.Sprite):
    def __init__(self, posicion):
        pygame.sprite.Sprite.__init__(self)
        # Cargar imagen
        self.image = pygame.image.load('ladrillo.png')
        # Obtener rectángulo de la imagen
        self.rect = self.image.get_rect()
        # Posición inicial, provista externamente.
        self.rect.topleft = posicion

class Muro(pygame.sprite.Group):
    def __init__(self, cantidadLadrillos):
        pygame.sprite.Group.__init__(self)

        pos_x = 0
        pos_y = 20
        for i in range(cantidadLadrillos):
            ladrillo = Ladrillo((pos_x, pos_y))
            self.add(ladrillo)

            pos_x += ladrillo.rect.width
            if pos_x >= ANCHO:
                pos_x = 0
                pos_y += ladrillo.rect.height

director = Director('Juego de ladrillos', (ANCHO, ALTO))
director.agregarEscena('Nivel1')
director.ejecutar('Nivel1')

In [ ]:
from tkinter import*
from tkinter import messagebox

libros=[]


def anadir():
    t=titulo.get()

    a=autor.get()

    e=editorial.get()

    np= NdePaginas.get()

    fl=fechalimite.get()

    libros.append(t+"$"+a+"$"+e+"$"+str(np)+"$"+fl)

    escribirLibro ()

    messagebox.showinfo ("Guardado", "El libro que tiene que leer ha sido guardado")

    titulo.set("")

    autor.set("")

    editorial.set("")

    NdePaginas.set("")

    fechalimite.set("")

    consultar()


def escribirLibro():

    archivo=open ("biblioteca.txt", "w")

    libros.sort()

    for elemento in libros:

        archivo.write(elemento+"\n")

    archivo.close()

def eliminarLibro():

    eliminado=eliminarlibro.get()

    removido= False

    for elemento in libros:

        arreglo = elemento.split("$")

        if eliminarlibro.get()==arreglo [0]:

            respuesta=messagebox.askyesno ("Eliminar libro", "Desea eliminar el libro con el titulo: \n"+eliminado)

            if respuesta:

                libros.remove (elemento)

                removido=True

                escribirLibro ()

                consultar()

                respuesta=""

        if removido:

            messagebox.showinfo ("Eliminar", "Se ha eliminado el tituno\n"+eliminado)

def salir ():
    sa=messagebox.askyesno("Salir","¿Deseas finalizar?")
    if sa==True:
        quit()

def iniciarArchivo():
    archivo = open("bibioteca.txt", "a")
    archivo.close()

def cargar():

    archivo=open("biblioteca.txt", "r")

    linea = archivo.readline ()

    if linea:

        while linea:

            if linea [-1]=='\n':

                linea=linea [:-1]

            libros.append(linea)

            linea=archivo.readline ()

        archivo.close()



def consultar():

    r=Text (ventana, width=80, height=15)

    libros.sort()

    valores=[]

    r.insert (INSERT, "\tTitulo del libro\n")

    r.insert (INSERT,"----------------------------------------\
--------------------------------------------------\n")

    for elemento in libros:

        arreglo=elemento.split("$")

        valores.append (arreglo [0])

        r.insert (INSERT, "\t"+arreglo [0] +"\n Autor: "+arreglo [1] +
                "\tEditorial: "+arreglo [2]+"\tN° de páginas: "+arreglo [3]+
                 "\tFecha limite:"+arreglo [4]+"\n")
                

        r.insert (INSERT, "--------------------------------------\
-----------------------------------------------------\n")

    spinTitulo=Spinbox (ventana, value= (valores),

        textvariable=eliminarlibro,width=70) .place (x=110, y=450)

    r.place (x=30,y=200)

    if libros==[]:

        spinTitulo= Spinbox (ventana, value=(valores), width=70) .place (x=110, y=450) 
        r.config (state=DISABLED)

ventana=Tk()
titulo=StringVar()
autor=StringVar()
editorial=StringVar()
NdePaginas=IntVar()
fechalimite=StringVar()
eliminarlibro=StringVar()
colorFondo="PaleVioletRed2"
colorLetra="#FFF"
iniciarArchivo()
cargar()
consultar()
ventana.title("Relación de Libros que tengo que leer")
ventana.geometry ("700x500")
ventana.resizable(0,0)
ventana.configure (background=colorFondo)
etiquetaTitulo =Label (ventana, text="Relación de libros\
    que tengo que leer", bg=colorFondo, fg= colorLetra) .place (x=250,y=10)

eTitulo=Label (ventana, text="Titulo: ", bg=colorFondo\
    ,fg= colorLetra) .place (x=30, y=40)

cTitulo =Entry (ventana, textvariable=titulo, width=50). place (x=120,y=40)
eAutor=Label (ventana, text="Autor: ", bg =colorFondo,\
    fg=colorLetra) .place (x=30,y=70)

cAutor=Entry (ventana, textvariable=autor). place (x=120,y=70)
eEditorial=Label (ventana, text="Editorial: ",\
    bg=colorFondo, fg = colorLetra) .place (x=30, y=100)

cEditorial=Entry (ventana, textvariable=editorial) .place (x=120,y=100) 
eNpaginas=Label (ventana, text="N° de páginas: ",\
    bg=colorFondo, fg=colorLetra) .place (x=30, y=130)

cNpaginas=Entry (ventana, textvariable=NdePaginas). place (x=123,y=130) 
eFechaLimite=Label (ventana, text="Fecha limite: ",\
    bg=colorFondo, fg=colorLetra). place (x=30, y=160) 
botonAnadir=Button (ventana, text="Añadir libro",\
    command=anadir, bg="#009", fg="white") .place (x=550,y=38)

cFechaLimite=Entry (ventana, textvariable=fechalimite)\
.place (x=120,y=160)

spinTitulo=Label (ventana, text="Titulo leido:",\
    bg= colorFondo, fg= colorLetra) .place (x=30,y=460) 
botonLeido=Button (ventana, text="Libro ya leido",\
    command=eliminarLibro, bg="#009",fg= "white") .place (x=550,y=470)

imgbtn=PhotoImage (file="salida.png")

sal =Button (ventana, image=imgbtn, command=salir) .place (x=370, y=80)

ventana.mainloop()